In [ ]:
print("hello world")

In [2]:
# --- Xử lý dữ liệu (CSV) ---
import pandas as pd
from langchain_core.documents import Document

# --- LangChain: Vector Store (Chroma) ---
from langchain_community.vectorstores import Chroma

# --- LangChain: Embedding (BAAI/bge-m3) ---
# Lớp bọc này sẽ sử dụng 'sentence-transformers' ở bên dưới
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- LangChain: LLM (Qwen2) ---
# Lớp bọc để tích hợp pipeline của Hugging Face vào LangChain
from langchain_community.llms import HuggingFacePipeline

# --- LangChain: Chain và Memory ---
# from langchain.chains import ConversationalRetrievalChain
# from langchain.memory import ConversationBufferMemory 

# --- Thư viện nền: Hugging Face (Transformers) & PyTorch ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, AutoModel


/root/Chatbot_LangChain_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
df = pd.read_csv('cleaned_data.csv')   
df.head()

,name,type,rating,count,address,comment,lat,lon,geometry
0,Tiệm cây Người Làm Vườn,Quán cà phê,4.7,87.0,"55 Sương Nguyệt Anh, Quận 1, Thành phố Hồ Chí ...",Do đi sáng chủ nhật nên không có mưa và không ...,10.771666,106.688863,POINT (106.6888632 10.7716658)
1,Tonkin Specialty Coffee,Quán cà phê,4.9,3600.0,"91 Lý Tự Trọng, Quận 1, Thành phố Hồ Chí Minh","Coffee ngon, chỗ ngồi chill và yên tĩnh, phù h...",10.774448,106.697708,POINT (106.697708 10.7744477)
2,See'm Coffee & Rooftop Quận 1,Quán cà phê,4.9,47.0,"85 Nguyễn Văn Thủ, Quận 1, Thành phố Hồ Chí Minh","nước ngon, nhân viên nhiệt tình, quán view đẹp",10.789077,106.698195,POINT (106.6981947 10.7890772)
3,The Balcony Coffee,Quán cà phê,4.8,174.0,"26 Lý Tự Trọng, Quận 1, Thành phố Hồ Chí Minh","Rất ấn tượng với món uống vừa ngon vừa đẹp, nê...",10.778196,106.701136,POINT (106.701136 10.7781958)
4,Soo Kafe Bến Thành,Quán cà phê,4.7,1800.0,"35 Phan Chu Trinh, Quận 1, Thành phố Hồ Chí Minh","Không gian vừa, đẹp, ngồi chill chill, hay làm...",10.772543,106.697155,POINT (106.6971552 10.7725434)


In [21]:
df.describe()

,rating,count,lat,lon
count,299.000000,290.000000,299.000000,299.000000
mean,4.222742,1366.141379,10.775307,106.698366
std,0.852760,5210.225018,0.007799,0.005723
min,0.000000,1.000000,10.758225,106.684093
25%,4.100000,87.000000,10.769416,106.694138
50%,4.300000,368.500000,10.774494,106.699658
75%,4.700000,1000.000000,10.778910,106.703141
max,5.000000,75000.000000,10.795748,106.707326


In [22]:
documents = []

for row in df.itertuples():
    content_to_embed = f"Tên quán: {row.name}. Loại hình: {row.type}. Bình luận: {row.comment}. Địa chỉ: {row.address}. Đánh giá {row.rating} sao từ {row.count} lượt đánh giá."
    
    # 2. TẠO METADATA (Thông tin để LỌC và Tham chiếu)
    # Đây là các dữ liệu "có cấu trúc" (rating, lat, lon)
    # Rất quan trọng: Chúng ta dùng metadata để LỌC (filter), KHÔNG phải để embed
    metadata = {
        "source_file": "cleaned_data.csv",
        "name": row.name,
        "type": row.type,
        "rating": row.rating,
        "rating_count": row.count,
        "address": row.address,
        "latitude": row.lat,
        "longitude": row.lon,
        "geometry": row.geometry
    }
    
    # 3. Tạo đối tượng Document (1 document cho mỗi hàng CSV)
    doc = Document(page_content=content_to_embed, metadata=metadata)
    documents.append(doc)

print(f"Da chuan bi xong {len(documents)} tai lieu.")


# -----------------------------------------------------------------
# BƯỚC 2: Khởi tạo Mô hình Embedding (BAAI/bge-m3)
# -----------------------------------------------------------------
print("Dang tai mo hinh embedding (BAAI/bge-m3) len GPU...")
# Lớp bọc 'HuggingFaceEmbeddings' sẽ tự động dùng 'sentence-transformers'
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'}, # Bắt buộc chạy trên GPU
    encode_kwargs={'normalize_embeddings': True} # Rất quan trọng cho BGE
)
print("Tai mo hinh embedding thanh cong!")


# -----------------------------------------------------------------
# BƯỚC 3: Add vào ChromaDB (Làm theo LÔ - BATCH)
# -----------------------------------------------------------------
# Đây là lúc nó chạy embedding cho tất cả 
# 'documents' và lưu vào Chroma
print("Bat dau qua trinh embedding (tren GPU) va luu vao ChromaDB...")
vector_store = Chroma.from_documents(
    documents=documents, 
    embedding=embeddings,
    persist_directory="./chroma_db_csv" # Thư mục để lưu trữ vĩnh viễn
)

print("\n--- HOAN TAT! ---")
print(f"Da embed va luu thanh cong {len(documents)} tai lieu vao thu muc './chroma_db_csv'")

# (Tùy chọn) Chạy thử nhanh
print("\n--- Chay thu tim kiem: ---")
query = "Quán cafe nào có bình luận yên tĩnh?"
search_results = vector_store.similarity_search(query, k=1)
print(f"Cau hoi: {query}")
print(f"Ket qua tim thay (Top 1):\n{search_results[0].page_content}")
print(f"Metadata (Rating): {search_results[0].metadata.get('rating')}")

Da chuan bi xong 299 tai lieu.
Dang tai mo hinh embedding (BAAI/bge-m3) len GPU...


/tmp/ipykernel_14222/337971234.py:33: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Tai mo hinh embedding thanh cong!
Bat dau qua trinh embedding (tren GPU) va luu vao ChromaDB...

--- HOAN TAT! ---
Da embed va luu thanh cong 299 tai lieu vao thu muc './chroma_db_csv'

--- Chay thu tim kiem: ---
Cau hoi: Quán cafe nào có bình luận yên tĩnh?
Ket qua tim thay (Top 1):
Tên quán: Memento Saigon. Loại hình: Quán cà phê. Bình luận: Toàn bộ không gian yên tĩnh, có khách chụp ảnh nhưng không gây ồn ào.. Địa chỉ: 14 Tôn Thất Đạm, Quận 1, Thành phố Hồ Chí Minh. Đánh giá 4.9 sao từ 189.0 lượt đánh giá.
Metadata (Rating): 4.9


In [31]:
query = "Tôi muốn đi xem thú?"
search_results = vector_store.similarity_search(query, k=2)
print(search_results[0].page_content)
print(search_results[1].page_content)

Tên quán: Vườn bướm. Loại hình: Vườn bách thú. Bình luận: Đây không chỉ là nơi để thư giãn mà còn là cơ hội học hỏi đầy ý nghĩa.. Địa chỉ: 2 Nguyễn Bỉnh Khiêm, Quận 1, Thành phố Hồ Chí Minh. Đánh giá 4.3 sao từ 24.0 lượt đánh giá.
Tên quán: The Monkey Gallery DINING. Loại hình: Món ăn sang trọng. Bình luận: 2 người ăn chọn  đơn ++  mà chưa no.. Địa chỉ: 3rd Floor, 91 Mạc Thị Bưởi, Quận 1, Thành phố Hồ Chí Minh. Đánh giá 4.5 sao từ 488.0 lượt đánh giá.


In [ ]:
# -----------------------------------------------------------------
# 1. Cấu hình Nén 4-bit (Quantization)
# -----------------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=True,
)

# -----------------------------------------------------------------
# 2. Tải Tokenizer và Model (Llama-3-8B-Instruct)
# -----------------------------------------------------------------
LLM_ID = "meta-llama/Meta-Llama-3-8B-Instruct" 
print(f"Dang tai Tokenizer {LLM_ID}...")
# Vì bạn đã đăng nhập HF, việc tải sẽ bắt đầu
tokenizer = AutoTokenizer.from_pretrained(LLM_ID, trust_remote_code=True) 

print("Dang tai Model Llama-3-8B (4-bit) len GPU...")
model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True 
)
print("Tai model Llama-3-8B thanh cong!")

# -----------------------------------------------------------------
# 3. Tạo Hugging Face Pipeline (Gói gọn quá trình tạo văn bản)
# -----------------------------------------------------------------
# Pipeline giúp quá trình gọi LLM dễ dàng hơn
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    pad_token_id=tokenizer.eos_token_id,
    # device=0 <--- XÓA DÒNG NÀY ĐI
)
print("Pipeline Llama-3-8B da duoc tao va san sang!")

/root/Chatbot_LangChain_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dang tai Model Qwen2 (4-bit) len GPU...


`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 4 files:   0%|          | 0/4 [03:46<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# (Giả sử các biến vector_store và text_generator đã được khởi tạo thành công)

# Khởi tạo Lịch sử (Memory)
chat_history = [] 

# --- 1. HÀM ĐỊNH DẠNG LỊCH SỬ ---
def format_history_for_llm(history: list) -> str:
    """Chuyển list history thành chuỗi văn bản cho Llama-3."""
    formatted = ""
    # Llama-3 (Instruct) thường dùng định dạng [INST] và [/INST]
    # Nhưng vì chúng ta đang nhồi vào phần HISTORY của SYSTEM PROMPT,
    # ta dùng định dạng User/Assistant cổ điển để dễ phân biệt.
    for role, message in history:
        # User/Assistant là định dạng dễ đọc nhất cho model
        formatted += f"{role.capitalize()}: {message}\n"
    return formatted

# --- 2. HÀM CHÍNH: RAG và GENERATION ---
def generate_rag_response(new_question: str, history: list, vector_store):
    
    # 1. Truy xuất (Retrieval) - Semantic Search
    # Lấy 3 tài liệu liên quan nhất từ ChromaDB (k=3)
    retrieved_docs = vector_store.similarity_search(new_question, k=3)
    
    # Lấy nội dung của các tài liệu
    context = "\n---\n".join([doc.page_content for doc in retrieved_docs])
    
    # 2. Định dạng Lịch sử
    history_text = format_history_for_llm(history)
    
    # 3. Tạo Prompt cuối cùng (System Prompt + Context + History + Question)
    # Chúng ta sử dụng cấu trúc Prompt tối ưu cho RAG:
    final_prompt = f"""
[SYSTEM PROMPT]
Bạn là một trợ lý AI tiếng Việt chuyên nghiệp.
Nhiệm vụ của bạn là phân tích LỊCH SỬ HỘI THOẠI và DỮ LIỆU CONTEXT (từ các quán ăn).
Trả lời câu hỏi mới của người dùng CHỈ DỰA TRÊN DỮ LIỆU CONTEXT CUNG CẤP.
Nếu thông tin không có trong CONTEXT, hãy trả lời: "Tôi xin lỗi, tôi không tìm thấy thông tin này trong dữ liệu quán ăn."

CONTEXT (Dữ liệu RAG được truy xuất):
---
{context}
---

LỊCH SỬ HỘI THOẠI:
{history_text}

CÂU HỎI MỚI: {new_question}

TRẢ LỜI CỦA BẠN:
"""
    
    # 4. Gọi LLM (Decoding)
    print("...Llama-3 dang suy luan...")
    output = text_generator(final_prompt, return_full_text=False)
    
    # Xử lý output để chỉ lấy phản hồi
    response = output[0]['generated_text'].strip()
    
    # 5. Cập nhật Lịch sử (Trước khi return)
    history.append(("user", new_question))
    history.append(("assistant", response))
    
    return response

# ----------------------------------------------------------------------
# CHẠY THỬ HỆ THỐNG (2 Lượt để test History)
# ----------------------------------------------------------------------
print("--- KHỞI ĐỘNG CHATBOT LAMA-3 RAG ---")

# Lượt 1: Câu hỏi RAG (Truy xuất)
question_1 = "Quán cà phê nào có bình luận nói rằng nó rất yên tĩnh và view đẹp?"
print(f"User 1: {question_1}")
answer_1 = generate_rag_response(question_1, chat_history, vector_store)
print(f"Bot: {answer_1}\n")

# Lượt 2: Câu hỏi theo dõi (Dùng History)
question_2 = "Quán đó tên gì và có phải là quán ăn không?"
print(f"User 2: {question_2}")
answer_2 = generate_rag_response(question_2, chat_history, vector_store)
print(f"Bot: {answer_2}")